<a href="https://colab.research.google.com/github/gummallaprasanth/DL---Sentiment-Analysis-on-IMDB-Dataset-of-Movie-Reviews/blob/main/DL_Sentiment_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis on IMDB Dataset of Movie Reviews

In [2]:
!pip install kaggle


### Import the dpendencies

In [6]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


Data Colletion from kaggle

In [7]:
# creating kaggle dictionary
kaggle_dictionary=json.load(open('/content/kaggle.json'))

In [11]:
# setup kaggle credential environment variables
os.environ['KAGGLE_USERNAME']=kaggle_dictionary['username']
os.environ['KAGGLE_KEY']=kaggle_dictionary['key']

In [12]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 22.3MB/s]
100% 25.7M/25.7M [00:02<00:00, 12.4MB/s]


In [14]:
# unzip the dataset file
with ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r') as zip:
  zip.extractall()

Loading the dataset

In [15]:
data=pd.read_csv('/content/IMDB Dataset.csv')

In [17]:
# print the first 5 rows of the data
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
# print the last 5 rows of the dataset
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [19]:
# checking the shape of the dataset

data.shape

(50000, 2)

50000 rows and 2 columns

In [20]:
# check the distrubution of data
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


Observation : the data is balanced so we can proceed with creating the neural network.

Now we can Replace the names with numericatl vaules in the output feature label encoding for sentiment feature.

Positve-->1

Negitive-->0

In [21]:
# Replacing the names with numerical values
data['sentiment']=data['sentiment'].replace({'positive':1,'negative':0})

In [23]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


Obsevation : sucessfully converted  the feature into numeric data

In [24]:
# split the data in to train data and test data
train_data , test_data=train_test_split(data,test_size=0.2,random_state=42)

In [25]:
# cheking the data shapes

train_data.shape , test_data.shape

((40000, 2), (10000, 2))

In [30]:
train_data.head()

,review,sentiment
39087,That's what I kept asking myself during the ma...,0
30893,I did not watch the entire movie. I could not ...,0
45278,A touching love story reminiscent of In the M...,1
16398,This latter-day Fulci schlocker is a totally a...,0
13653,"First of all, I firmly believe that Norwegian ...",0


Data Preprocessing

Tokenization is the process of breaking down text into smaller units called tokens. These tokens can be words, subwords, or characters.

Tokenization is an important step in natural language processing (NLP) because it allows computers to work with text in a more structured way. By breaking down text into smaller units, computers can more easily analyze the relationships between words and phrases, and they can also learn to represent words and phrases in a numerical format that can be used for machine learning.

In [31]:
# Tokenize the data

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)


Observation : In the above process we do first getting the most repeated words from the reviews data with help of the tokenizer and converting into the numeric values in some particular sequences.

The second one in above process that we are using the Pad_sequences it is used for the maiantain  the text into the same dimensions that the maximum length should be 200 ,beacuse our model will understand well if the data is in the Uniform dimensions Like,when doing image classifcation we are making the image dimesnions in the same format like 28 x 28.  

In [33]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [32]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

Obsevation : sucessfully we converted into the text data to numerical data

In [34]:
# creating the  labels for y_train and y_test
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [37]:
y_test.shape , y_train.shape

((10000,), (40000,))

In [38]:
y_test

,sentiment
33553,1
9427,1
199,0
12447,1
39489,0
...,...
28567,0
25079,1
18707,1
15200,0


# LSTM---> Long Short Term Memory

### Build the Model

LSTMs address the vanishing gradient problem that can occur with traditional RNNs, allowing them to learn long-term dependencies in data.

They are used in the code you provided to analyze movie review text and predict sentiment (positive or negative).

In [48]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim =128, input_length=200)) # Embedding layer
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) # LSTM layer
model.add(Dense(1, activation='sigmoid')) # output layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


5000 is the input dimensions came from tokenizer , 128 is the vectorspace for the text data, dropout is used to overfit the model some neurons will flow as 0 data. the dence layer is the output layer previous layers add to the output layes and give the oytput beacuse it is binary classification.

we used sigmoid activation beacuse our output should be in the range of 0 to 1 so if the values is >0.5 it represent as the 1 and  < 0.5 represent as the  0 output in the results.

In [ ]:
# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training the model

In [50]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 76s 271ms/step - accuracy: 0.7033 - loss: 0.5526 - val_accuracy: 0.8500 - val_loss: 0.3502
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 74s 262ms/step - accuracy: 0.8568 - loss: 0.3504 - val_accuracy: 0.8389 - val_loss: 0.3698
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 85s 273ms/step - accuracy: 0.8762 - loss: 0.3045 - val_accuracy: 0.8643 - val_loss: 0.3320
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 93s 318ms/step - accuracy: 0.8925 - loss: 0.2757 - val_accuracy: 0.8459 - val_loss: 0.3632
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 67s 259ms/step - accuracy: 0.8699 - loss: 0.3160 - val_accuracy: 0.8690 - val_loss: 0.3195
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 65s 259ms/step - accuracy: 0.9026 - loss: 0.2511 - val_accuracy: 0.8553 - val_loss: 0.3400
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 258ms/step - accuracy: 0.9101 - loss: 0.2253 - val_accuracy: 0.8710 - val_loss: 0.3201
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 90s 288ms/step - accuracy: 0.9261 - loss: 0

If we use more epoches it will take more time use at medium level number of Epoches.

### Model Evaluation

In [51]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 84ms/step - accuracy: 0.8730 - loss: 0.3545
Test loss: 0.35416489839553833
Test accuracy: 0.8726000189781189


# Building a Predictive Model

In [68]:
def predictive_sentiment(review):
  # tokenize and pad the review
    review = tokenizer.texts_to_sequences([review])
    review = pad_sequences(review, maxlen=200)
    prediction = model.predict(review)
    if prediction[0][0] > 0.5:
        return 'Positive'
    else:
        return 'Negative'


input_review="the movie is fantastic.i loved it"
sentiment=predictive_sentiment(input_review)
print("the sentiment of the review: ",sentiment)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
the sentiment of the review:  Positive


It works well the model will predict the review output correctly

In [69]:
# use for some other examples
input_review="the movie was not that good"
sentiment=predictive_sentiment(input_review)
print("the sentiment of the review: ",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
the sentiment of the review:  Negative


In [70]:
input_review="the movie is very bad "
sentiment=predictive_sentiment(input_review)
print("the sentiment of the review: ",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
the sentiment of the review:  Negative
